In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Normalization
from keras.models import Model
from sklearn.metrics import accuracy_score

%matplotlib inline

2023-04-29 11:05:24.838423: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-29 11:05:24.866707: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-29 11:05:24.867158: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 11:05:26.265262: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Preprocessing

In [2]:
df = pd.read_csv("data/Custom_Dataset.csv")

In [3]:
# Label 0 for benign and 1 for any attack
df["Label1"] = np.where(df[' Label']==0, 0, 1)


# Middle Label SQLi(10) as 1,MITM(11) as 2, Dictionary(12) as 3 and all Ddos as 0 
conditions = [
    (df[' Label'] == 10),
    (df[' Label'] == 11),
    (df[' Label'] == 12),
    (df[' Label'] == 0),
    df[' Label'].isin([1,2,3,4,5,6,7,8,9,13])]
choices = [1, 2, 3, 4, 0]
df['Label2'] = np.select(conditions, choices)


# Innermost Label 0 tftp(1) 1 udp+ssdp(2) 2 ldap(3) 3 netport(4)
# 4 syn(5) 5 mssql(6) 6 hulk(7) 7 rudy(8) 8 goldeneye(9) 9 httpbypass(13)
conditions = [
    (df[' Label'] == 1),
    (df[' Label'] == 2),
    (df[' Label'] == 3),
    (df[' Label'] == 4),
    (df[' Label'] == 5),
    (df[' Label'] == 6),
    (df[' Label'] == 7),
    (df[' Label'] == 8),
    (df[' Label'] == 9),
    (df[' Label'] == 13),
    (df[' Label'].isin([0,10,11,12]))
]
choices = [0,1,2,3,4,5,6,7,8,9,10]
df['Label3'] = np.select(conditions, choices)

# Layer 1 

In [4]:
X = df[[' Source Port', ' Destination Port', ' Protocol', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Fwd Packet Length Min',
       'Bwd Packet Length Max', ' Flow IAT Mean', ' Flow IAT Std',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Min',
       ' Fwd Header Length', ' Bwd Header Length', ' Bwd Packets/s',
       ' Init_Win_bytes_backward', ' NFlow Packets/s']]
y = df['Label1']

In [5]:
oversample = SMOTE()
X,y = oversample.fit_resample(X,y)

In [6]:
y.value_counts()

Label1
1    1838583
0    1838583
Name: count, dtype: int64

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30, random_state=42)

In [8]:
y_test.value_counts()

Label1
0    551770
1    551380
Name: count, dtype: int64

In [9]:
normalizer1 = Normalization(axis=-1)
normalizer1.adapt(np.array(X_train))

2023-04-29 10:42:37.933841: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-29 10:42:37.934208: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-04-29 10:42:38.003608: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205921280 exceeds 10% of free system memory.


In [10]:
input_layer = Input(shape=(20,))
normal1 = normalizer1(input_layer) 
layer11 = Dense(7,activation='relu')(normal1)
layer12 = Dense(3,activation='relu')(layer11)
output1 = Dense(1,activation='sigmoid')(layer12)

model1 = Model(inputs = input_layer, outputs = output1, name = 'model1')
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 normalization (Normalizatio  (None, 20)               41        
 n)                                                              
                                                                 
 dense (Dense)               (None, 7)                 147       
                                                                 
 dense_1 (Dense)             (None, 3)                 24        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 216
Trainable params: 175
Non-trainable params: 41
_____________________________________________________________

In [11]:
model1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model1.fit(X_train,y_train,validation_split=0.3,epochs=20,verbose=1,batch_size=10000)

Epoch 1/20


2023-04-29 10:43:12.186902: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 288289760 exceeds 10% of free system memory.


181/181 [==============================] - 1s 5ms/step - loss: 0.5034 - accuracy: 0.8186 - val_loss: 0.4098 - val_accuracy: 0.8516
Epoch 2/20
  1/181 [..............................] - ETA: 0s - loss: 0.4192 - accuracy: 0.8450

2023-04-29 10:43:13.349518: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 123552800 exceeds 10% of free system memory.


181/181 [==============================] - 1s 3ms/step - loss: 0.3702 - accuracy: 0.8605 - val_loss: 0.3378 - val_accuracy: 0.8710
Epoch 3/20
181/181 [==============================] - 0s 3ms/step - loss: 0.3087 - accuracy: 0.8898 - val_loss: 0.2809 - val_accuracy: 0.9045
Epoch 4/20
181/181 [==============================] - 1s 3ms/step - loss: 0.2564 - accuracy: 0.9085 - val_loss: 0.2342 - val_accuracy: 0.9141
Epoch 5/20
181/181 [==============================] - 1s 3ms/step - loss: 0.2011 - accuracy: 0.9267 - val_loss: 0.1673 - val_accuracy: 0.9492
Epoch 6/20
181/181 [==============================] - 0s 3ms/step - loss: 0.1460 - accuracy: 0.9520 - val_loss: 0.1278 - val_accuracy: 0.9548
Epoch 7/20
181/181 [==============================] - 1s 3ms/step - loss: 0.1121 - accuracy: 0.9610 - val_loss: 0.0988 - val_accuracy: 0.9705
Epoch 8/20
181/181 [==============================] - 1s 3ms/step - loss: 0.0889 - accuracy: 0.9754 - val_loss: 0.0810 - val_accuracy: 0.9784
Epoch 9/20
181/18

In [15]:
from sklearn.metrics import accuracy_score
import seaborn as sns

layer1pred = model1.predict(X_test)


n1 = []
for i in range(len(layer1pred)):
  if layer1pred[i] >= 0.5:
    n1.append(1)
  else:
    n1.append(0)

accuracylayer1 = accuracy_score(y_test, n1)

print("Layer 1 accuracy: ",accuracylayer1)

  244/34474 [..............................] - ETA: 14s 

2023-04-29 10:47:37.497296: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 176504000 exceeds 10% of free system memory.


34474/34474 [==============================] - 13s 376us/step
Layer 1 accuracy:  0.9924207949961474


In [13]:
model1.save('layer1.h5')

# Layer 2

In [9]:
df2 = df[df["Label2"] != 4]
y = df2['Label2']
X = df2[[' Source Port', ' Destination Port', ' Protocol', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Fwd Packet Length Min',
       'Bwd Packet Length Max', ' Flow IAT Mean', ' Flow IAT Std',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Min',
       ' Fwd Header Length', ' Bwd Header Length', ' Bwd Packets/s',
       ' Init_Win_bytes_backward', ' NFlow Packets/s']]

In [10]:
oversample = SMOTE()
X,y = oversample.fit_resample(X,y)

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30, random_state=42)

In [12]:
y_test.value_counts()

Label2
3    534348
0    533815
1    533439
2    533096
Name: count, dtype: int64

In [13]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')
#perform one-hot encoding on 'team' column 
encoding = pd.DataFrame(encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray())
encoding_test = pd.DataFrame(encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray())

In [14]:
normalizer2 = Normalization(axis=-1)
normalizer2.adapt(np.array(X_train))

2023-04-29 10:56:55.168633: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-29 10:56:55.169250: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-04-29 10:56:55.973508: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 398476960 exceeds 10% of free system memory.


In [15]:
input_layer = Input(shape=(20,))
normal2 = normalizer2(input_layer)
layer21 = Dense(6,activation='relu')(normal2)
layer22 = Dense(4,activation='relu')(layer21)
output2 = Dense(4,activation='softmax')(layer22)

model2 = Model(inputs = input_layer, outputs = output2, name = 'model2')
model2.summary()



Model: "model2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 normalization (Normalizatio  (None, 20)               41        
 n)                                                              
                                                                 
 dense (Dense)               (None, 6)                 126       
                                                                 
 dense_1 (Dense)             (None, 4)                 28        
                                                                 
 dense_2 (Dense)             (None, 4)                 20        
                                                                 
Total params: 215
Trainable params: 174
Non-trainable params: 41
_____________________________________________________________

In [19]:
model2.compile(optimizer='adam', 
          loss='categorical_crossentropy', 
          metrics=['accuracy'])
  
model2.fit(X_train,encoding,epochs=20,verbose=1,batch_size=10000)

Epoch 1/20


2023-04-29 10:59:41.620639: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 796953920 exceeds 10% of free system memory.
2023-04-29 10:59:42.052257: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 159390784 exceeds 10% of free system memory.


499/499 [==============================] - 3s 3ms/step - loss: 0.7653 - accuracy: 0.7859
Epoch 2/20
499/499 [==============================] - 2s 3ms/step - loss: 0.1907 - accuracy: 0.9523
Epoch 3/20
499/499 [==============================] - 2s 3ms/step - loss: 0.1021 - accuracy: 0.9706
Epoch 4/20
499/499 [==============================] - 2s 3ms/step - loss: 0.0738 - accuracy: 0.9833
Epoch 5/20
499/499 [==============================] - 2s 3ms/step - loss: 0.0587 - accuracy: 0.9854
Epoch 6/20
499/499 [==============================] - 2s 3ms/step - loss: 0.0496 - accuracy: 0.9875
Epoch 7/20
499/499 [==============================] - 2s 3ms/step - loss: 0.0420 - accuracy: 0.9906
Epoch 8/20
499/499 [==============================] - 2s 3ms/step - loss: 0.0374 - accuracy: 0.9921
Epoch 9/20
499/499 [==============================] - 2s 3ms/step - loss: 0.0337 - accuracy: 0.9936
Epoch 10/20
499/499 [==============================] - 2s 3ms/step - loss: 0.0297 - accuracy: 0.9943
Epoch 11/2

In [23]:
layer2pred = model2.predict(X_test)

n2 = []
for i in layer2pred:
  i=list(i)
  maxvalue=max(i)
  n2.append(i.index(maxvalue))

accuracylayer2 = accuracy_score(y_test, n2)

print("Layer 2 accuracy: ",accuracylayer2)


66710/66710 [==============================] - 25s 381us/step
Layer 2 accuracy:  0.9955811079600019


In [21]:
model2.save('layer2.h5')

# Layer 3

In [4]:
df3 = df[df["Label3"] != 10]
y3 = df3['Label3']
X3 = df3[[' Source Port', ' Destination Port', ' Protocol', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Fwd Packet Length Min',
       'Bwd Packet Length Max', ' Flow IAT Mean', ' Flow IAT Std',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Min',
       ' Fwd Header Length', ' Bwd Header Length', ' Bwd Packets/s',
       ' Init_Win_bytes_backward', ' NFlow Packets/s']]

In [5]:
oversample = SMOTE()
X3,y3 = oversample.fit_resample(X3,y3)

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X3,y3,test_size=0.30, random_state=42)

In [7]:
y_test.value_counts()

Label3
2    122206
5    122132
6    121867
4    121731
3    121697
0    121599
8    121573
7    121489
1    121331
Name: count, dtype: int64

In [9]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')
#perform one-hot encoding on 'team' column 
encoding = pd.DataFrame(encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray())
encoding_test = pd.DataFrame(encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray())

In [10]:


normalizer3 = Normalization(axis=-1)
normalizer3.adapt(np.array(X_train))




input_layer = Input(shape=(20,))
normal3 = normalizer3(input_layer)
layer31 = Dense(9,activation='relu')(normal3)
layer32 = Dense(9,activation='relu')(layer31)
layer33 = Dense(9,activation='relu')(layer32)
output3 = Dense(9,activation='softmax')(layer33)
model3 = Model(inputs = input_layer, outputs = output3, name = 'model3')
model3.summary()
model3.compile(optimizer='adam', 
          loss='categorical_crossentropy', 
          metrics=['accuracy'])
  
model3.fit(X_train,encoding,epochs=20,verbose=1,batch_size=10000)




2023-04-29 11:10:21.533251: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-29 11:10:21.533577: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "model3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 normalization (Normalizatio  (None, 20)               41        
 n)                                                              
                                                                 
 dense (Dense)               (None, 9)                 189       
                                                                 
 dense_1 (Dense)             (None, 9)                 90        
                                                                 
 dense_2 (Dense)             (None, 9)                 90        
                                                                 
 dense_3 (Dense)             (None, 9)                 90        
                                                            

2023-04-29 11:10:45.814697: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 409033280 exceeds 10% of free system memory.


256/256 [==============================] - 2s 4ms/step - loss: 1.4829 - accuracy: 0.4432
Epoch 2/20
256/256 [==============================] - 1s 4ms/step - loss: 0.5571 - accuracy: 0.8515
Epoch 3/20
256/256 [==============================] - 1s 4ms/step - loss: 0.3300 - accuracy: 0.9063
Epoch 4/20
256/256 [==============================] - 1s 4ms/step - loss: 0.2350 - accuracy: 0.9270
Epoch 5/20
256/256 [==============================] - 1s 4ms/step - loss: 0.1905 - accuracy: 0.9395
Epoch 6/20
256/256 [==============================] - 1s 4ms/step - loss: 0.1655 - accuracy: 0.9475
Epoch 7/20
256/256 [==============================] - 1s 4ms/step - loss: 0.1474 - accuracy: 0.9541
Epoch 8/20
256/256 [==============================] - 1s 4ms/step - loss: 0.1315 - accuracy: 0.9604
Epoch 9/20
256/256 [==============================] - 1s 4ms/step - loss: 0.1188 - accuracy: 0.9643
Epoch 10/20
256/256 [==============================] - 1s 4ms/step - loss: 0.1081 - accuracy: 0.9676
Epoch 11/2

In [15]:
layer3pred = model3.predict(X_test)

n3 = []
for i in layer3pred:
  i=list(i)
  maxvalue=max(i)
  n3.append(i.index(maxvalue))

accuracylayer3 = accuracy_score(y_test, n3)

print("Layer 3 accuracy: ", accuracylayer3)

34239/34239 [==============================] - 13s 382us/step
Layer 3 accuracy:  0.9856867084997147


In [14]:
model3.save('layer3.h5')